<a href="https://colab.research.google.com/github/Rogerio-mack/IMT_CD_2024/blob/main/IMT_CV_GridSearchCV_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# Cross Validation a GridSearchCV

[Alternative scikit-learn course](https://inria.github.io/scikit-learn-mooc/toc.html)

In [1]:
from IPython.display import IFrame

IFrame('https://allisonhorst.github.io/palmerpenguins/', width=800, height=600)


In [2]:
import pandas as pd
import seaborn as sns
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# Dados

In [3]:
df = sns.load_dataset('penguins')
df.dropna(inplace=True)

df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male


# Treinamento e Teste

In [4]:
# As features (X) e o alvo (y)
X = df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]
y = df['species']

# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Modelos

In [5]:
# Criando o dicionário de modelos que iremos testar
models = {
    'RandomForest': RandomForestClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'KNN': KNeighborsClassifier()
}

# Cross Validation Scores


## Exercício 1.
O obtenha a acuracidade média dos modelos para um cv de 5 partições. Explore a saída do estimador `cross_val_score()`.

In [6]:
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    print(f"{name}, score = {scores.mean():.4f}")

RandomForest, score = 0.9790
DecisionTree, score = 0.9639
LogisticRegression, score = 0.9850
KNN, score = 0.8022


In [7]:
scores

array([0.71641791, 0.7761194 , 0.79104478, 0.81818182, 0.90909091])

## Exercício 2.
Acima foram empregadas partições diferentes para cada modelo. Empregue os estimadores KFold (ou alternativamente o StratifiedKFold), para fixar as partições e obtenha o novo score dos modelos fazendo a seleção agora pela métrica `f1_macro`.

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=kf, scoring='f1_macro')
    print(f"{name}, score = {scores.mean():.4f}")

RandomForest, score = 0.9760
DecisionTree, score = 0.9567
LogisticRegression, score = 0.9826
KNN, score = 0.6891


## Exercício 3.
Empregue o código anterior para selecionar *programaticamente* o melhor modelo, treinar, aplicar ao conjunto de teste e obter a acuracidade nesse conjunto.

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=1)

models_scores = {}

for name, model in models.items():
    models_scores[model] = cross_val_score(model, X, y, cv=kf, scoring='f1_macro').mean()

models_scores

{RandomForestClassifier(): 0.9691534609181669,
 DecisionTreeClassifier(): 0.9662791535762952,
 LogisticRegression(max_iter=1000): 0.9826031154108422,
 KNeighborsClassifier(): 0.6890947070491956}

In [10]:
best_model = max(models_scores, key=models_scores.get)
print(best_model)


LogisticRegression(max_iter=1000)


In [11]:
model = best_model
y_pred = model.fit(X_train,y_train).predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy:.2f}")

Acurácia: 0.99


In [12]:
model.get_params()


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

## Exercício 4.
Empregando o `GridSearchCV`. O `GridSearchCV` permite automatizar todas essas operações.

Empregue o exemplo de código abaixo para corrigir, como fizemos antes, o uso de partições diferentes na avaliação dos modelos (use o `KFold`).

In [13]:
# Criando o pipeline com pré-processamento e modelo
pipeline = Pipeline([('model', None)])
# pipeline = Pipeline([('scaler', StandardScaler()), ('model', None)])

# Definindo o dicionário de parâmetros para o GridSearchCV (somente diferentes modelos)
param_grid = [
    {'model': [models['RandomForest']]},
    {'model': [models['DecisionTree']]},
    {'model': [models['LogisticRegression']]},
    {'model': [models['KNN']]}
]

# Usando o GridSearchCV para encontrar o melhor modelo
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
# grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')

grid_search.fit(X_train, y_train)

# Exibindo o melhor modelo encontrado
print(f"Melhor modelo: {grid_search.best_estimator_['model']}")

# Avaliando o desempenho no conjunto de teste
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no teste: {accuracy:.2f}")


Melhor modelo: LogisticRegression(max_iter=1000)
Acurácia no teste: 0.99


In [27]:
# Criando o pipeline com pré-processamento e modelo
pipeline = Pipeline([('model', None)])
# pipeline = Pipeline([('scaler', StandardScaler()), ('model', None)])

# Definindo o dicionário de parâmetros para o GridSearchCV (somente diferentes modelos)
param_grid = [
    {'model': [models['RandomForest']]},
    {'model': [models['DecisionTree']], 'model__max_depth': [5,6,7]},
    {'model': [models['LogisticRegression']]},
    {'model': [models['KNN']], 'model__n_neighbors': [3, 5, 7, 9]}
]

kf = KFold(n_splits=5, shuffle=True, random_state=1)

# Usando o GridSearchCV para encontrar o melhor modelo
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='accuracy')
# grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='f1_macro')

grid_search.fit(X_train, y_train)

# Exibindo o melhor modelo encontrado
print(f"Melhor modelo: {grid_search.best_estimator_['model']}")

# Avaliando o desempenho no conjunto de teste
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia no teste: {accuracy:.2f}")


Melhor modelo: LogisticRegression(max_iter=1000)
Acurácia no teste: 0.99
